In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


## Create image augmentation

In [2]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',    
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    'data/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 8185 images belonging to 120 classes.
Found 2037 images belonging to 120 classes.


## Convolutional Network

In [3]:
model = Sequential()

# convolutional layer
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# fully connected layer for categorical clasification
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))  # last layer total class

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## Train network

In [4]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    mode='auto', 
    verbose=1)

callbacks = [early_stopping]


model.fit_generator(
    train_generator,
    steps_per_epoch=8185 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=2037 // batch_size,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
512/511 [==============================] - 353s 689ms/step - loss: 4.7977 - acc: 0.0080 - val_loss: 4.7818 - val_acc: 0.0123
Epoch 2/50
373/511 [====================>.........] - ETA: 1:26 - loss: 4.7836 - acc: 0.0121

KeyboardInterrupt: 